In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [10]:

def to_dataframe(json_data):
    df = pd.DataFrame(json_data, columns=['OpenTime', 'Open', 'High', 'Low', 'Close', 'Volume', 'CloseTime', 'BaseAssetVolume', 'NbTrades', 'TakerBuyVol', 'TakerBuyBaseAssetVol', 'Ignore'])
    df.drop(columns=['CloseTime', 'BaseAssetVolume', 'NbTrades', 'TakerBuyVol', 'TakerBuyBaseAssetVol', 'Ignore'], inplace=True)
    df['open_time'] = pd.to_datetime(df['OpenTime'], unit='ms')
    df.set_index(keys=['open_time'], inplace=True)

    df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']].astype('float32')
    df.columns = [col.lower() for col in df.columns]

    return df


def get_klines(base_url, symbol, interval='1m'):
    end_time = int(datetime.utcnow().timestamp() * 1000)
    url = f"{base_url}/klines?symbol={symbol}&limit=1000&interval={interval}&endTime={end_time}"

    headers = {
        'Content-Type': 'application/json'
    }

    response = requests.request("GET", url, headers=headers)

    df = to_dataframe(response.json())
    return df


def get_futures_data(symbol, interval='1m'):
    return get_klines('https://fapi.binance.com/fapi/v1', symbol, interval=interval)



def get_coinm_data(symbol, interval='1m'):
    # 
    return get_klines('https://dapi.binance.com/dapi/v1', symbol, interval=interval)


def get_spot_data(symbol, interval='1m'):
    return get_klines('https://api.binance.com/api/v3', symbol, interval=interval)


In [11]:
# symbols = pd.read_csv(f'{DATA_PATH}/../symbols.csv')['symbol'].values.tolist()
symbols = ['BTC', 'ETH', 'BNB', 'XRP', 'ADA', 'SOL', 'LUNA', 'AVAX', 'DOT', 'DOGE', 'SHIB', 'UST', 'MATIC', 'CRO', 'WBTC', 'DAI', 'ATOM', 'LTC', 'LINK']
symbols = [f'{s}USDT' for s in symbols]
DATA_PATH = '/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/crypto'
TF = '1m'


data = []

for symbol in symbols:
    df = get_futures_data(symbol=symbol, interval=TF)
    df.to_csv(f'{DATA_PATH}/{TF}/{symbol}.csv')

